In [1]:
region_name = "ALL_10-800_LOOSE"#"TDS" # _ELAISN1
probabilities = [0.000063342484]  # 3 sigma 0.002699796063, 4 sigma 0.000063342484
srcs_ids = [8215 ,32448 ,54184 ,98746 ,151796 ,166179 ,172775 ,193067 ,199832 ,259271 ,265150 ,297278 ,305192 ,388172 ,432606 ,472623 ,535864 ,541266 ,625693 ,1521738 ,1551422 ,2136829 ,] # WD ALL_10-800-LOOSE

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import time

from regions import CircleSkyRegion
from astropy import units as uu
from astropy.coordinates import SkyCoord
from astropy.wcs.utils import pixel_to_skycoord
from astropy.timeseries import LombScargle,TimeSeries,BinnedTimeSeries
from astropy.units import cds
from astropy.table import QTable
from astropy.time import Time

from loguru import logger
logger.enable("vasca")

#cds.enable()  
import numpy as np

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis

from vasca.utils import mag2flux, sel_sources, dd_filter2id

region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
rg = Region()
rg.load_from_fits(region_fname)
#rg.remove_table("tt_coadd_detections")
#rg.tt_coadd_sources.remove_columns(["nr_det","obs_filter_id","flux_nxv","flux_var","flux_cpval","flux_rchiq","fd_src_id","assoc_id","assoc_dist","assoc_ffactor","assoc_fdiff_s2n","rg_fd_id","nr_fd_dets","pos_xv","pos_var","pos_cpval","pos_rchiq"])
#rg.tt_sources.remove_columns(["fd_src_id","rg_fd_id"])
#rg.write_to_fits(region_fname)
rg.tt_sources.add_index("rg_src_id")
src_idx = rg.tt_sources.loc_indices[srcs_ids]
tt_src = rg.tt_sources[src_idx]
display(tt_src)

fd_src_id,nr_det,ra,dec,pos_err,pos_xv,pos_var,pos_cpval,pos_rchiq,coadd_src_id,coadd_dist,obs_filter_id,sel,flux,flux_err,flux_nxv,flux_var,flux_cpval,flux_rchiq,coadd_ffactor,coadd_fdiff_s2n,rg_fd_id,rg_src_id,nr_fd_srcs,hr,hr_err
,,deg,deg,arcsec,arcsec2,arcsec2,,,,arcsec,,,1e-06 Jy,1e-06 Jy,,1e-12 Jy2,,,,,,,,,
int32,int32[2],float64,float64,float32,float32,float32,float32,float32,int64,float32,int32[2],bool,float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],float32,float32,int64,int32,int32,float32,float32
-1,67 .. 39,35.24181331249359,-5.3268123630922135,0.06502271,-0.3229914,0.15286975,1.0,0.28614402,816,0.12559491,1 .. 2,True,15.8211355 .. 6.618499,0.10766479 .. 0.13884212,0.018202152 .. 0.01643959,5.4822598 .. 1.6915867,0.0 .. 0.00016123278,7.0483823 .. 2.0399196,-100.0,-10000.0,-1,8215,6,0.40061885,0.009090735
-1,43 .. 3,160.98628310403117,58.12556445357027,0.074825816,-0.0422247,0.21750309,0.09029268,0.8487536,51761,0.08939208,1 .. 2,True,200.61136 .. 207.43813,0.38346985 .. 2.5625901,0.058434978 .. 0.021001792,2359.211 .. 923.8086,0.0 .. 7.736314e-22,293.03583 .. 48.610947,-100.0,-10000.0,-1,32448,4,1.2079993,0.017565036
-1,46 .. 5,9.974461149474259,40.66514017555066,0.08865072,-0.022966983,0.38019085,0.33848307,0.9459373,-1,-1.0,1 .. 2,True,60.77122 .. 58.39467,0.21031997 .. 1.2000568,0.00084997487 .. 0.0030169173,5.252687 .. 17.953432,3.7028865e-08 .. 0.01857928,2.5735614 .. 2.9600725,-100.0,-10000.0,-1,54184,4,0.9690136,0.027690087
-1,4 .. 1,149.50430287014058,2.213943745696226,0.3462598,0.47544754,1.4778644,0.11377874,1.0737575,844112,0.40557882,1 .. 2,True,3.902144 .. 17.025454,0.2973466 .. 1.3194003,34.15577 .. -100.0,520.93 .. -100.0,0.0 .. -1.0,263.5419 .. -1.0,-100.0,-10000.0,-1,98746,3,0.35459045,0.029906916
-1,15 .. 4,348.9035688841206,19.413759373254578,0.12630433,-0.16186069,0.17914553,0.95801276,0.40485027,118292,0.15854242,1 .. 2,True,126.54993 .. 38.594788,0.6038925 .. 1.9439608,0.0017923283 .. 0.0030419875,35.589436 .. 59.990253,7.254839e-15 .. 0.45650917,7.074004 .. 0.86853236,-100.0,-10000.0,-1,151796,2,0.31256443,0.016097367
-1,7 .. 2,334.6190008630954,-0.00334150877900888,0.18584667,-0.18340522,0.12893102,0.8110655,0.40349314,-1,-1.0,1 .. 2,True,121.26045 .. 42.269413,0.7279036 .. 0.8880146,0.0016065001 .. 0.005644754,27.450546 .. 11.66761,2.65163e-07 .. 0.00661471,6.8656754 .. 7.3747196,-100.0,-10000.0,-1,166179,2,0.35823846,0.011588516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-1,12 .. 10,49.89487839440356,41.93511662149011,0.11688124,-0.13351366,0.18084738,0.7082775,0.6558279,995093,0.14107761,1 .. 2,True,146.99852 .. 48.611095,0.724471 .. 0.7923874,0.0026083186 .. 0.003205992,63.340225 .. 14.719814,7.10371e-20 .. 0.005771271,10.640779 .. 2.5773094,-100.0,-10000.0,-1,472623,2,0.3276734,0.0056179515


In [3]:
#dd_lcs = rg.get_light_curve(rg_src_ids=srcs_ids)

def freq2period(ff):
    return 1/ff

def period2freq(pp):
    return 1/pp

def plot_LombScargle(rg_src_id):

    t_start = time.time()

    #Get source and light curve
    tc_src = rg.get_src_from_id(rg_src_id, load_from_file = False, write_to_file=False, add_sed=False, add_gphoton=False, add_spectrum=False)
    lc_orig = tc_src.tt_source_lc
    sel_lc = np.array((lc_orig["obs_filter_id"]==1) * (lc_orig["sel"]==True), dtype=bool)
    lc = lc_orig[sel_lc]

    #Get LS parameters
    dt = lc["time"][1:]-lc["time"][0:-1]
    t_min = np.min(lc["time"].quantity)
    t_max = np.max(lc["time"].quantity)
    dt_tot =  t_max - t_min
    dt_min = np.min(dt.quantity)
    f_min = 1/(dt_tot/4)
    f_max = 1/(4*dt_min)

    # Run LS
    
    t_ls_start = time.time()
    print("Loading:",t_ls_start - t_start)
    ls = LombScargle(lc["time"], lc["flux"],lc["flux_err"]) #normalization{‘standard’, ‘model’, ‘log’, ‘psd’},
    freq,power = ls.autopower(minimum_frequency=f_min,maximum_frequency=f_max) #nyquist_factor=5
    t_ls_done = time.time()
    print("LS calculation:",t_ls_done - t_ls_start)

    
    #Get confidence interval
    #conf = ls.false_alarm_level(probabilities)

    #Get some basic results
    p_peak = power.max()
#    idx_pmax =  #np.where(power == p_peak)
    f_peak = freq[np.argmax(power)]
    Pval =ls.false_alarm_probability(p_peak)
    t_false_prob = time.time()
    print("False prob. calc:",t_false_prob-t_ls_done)
    print("Total:",t_false_prob - t_start)

    #Print basic info
    print(f"Lightcurve total duration= {dt_tot} ({1/dt_tot}), minimum bin distance {dt_min} ({1/dt_min})")
    print(f"Freq_min= {freq[0]}, freqmax= {freq[-1]}, number of freq bins {len(freq)}")
    print(f"Maximum power at frequency {f_peak }, period {1/f_peak}, whith a false probability of {Pval}")
    
    #Plot power spectrum
    fig, axs = plt.subplots(1, 2, figsize=(14, 6), num="LombScargle", clear=True)
    axs = axs.flatten()
    #axs[0].axhline(conf[0],linewidth=0.5, ls="--", color='k', label="4-sigma")
    axs[0].plot(freq, power)

    axs[0].set_xscale("log")
    axs[0].set_xlabel('Frequency [1/day]')
    axs[0].set_ylabel('LS power')
    secax = axs[0].secondary_xaxis("top", functions=(freq2period, period2freq))
    secax.set_xlabel("Period [day]")
    axs[0].legend()
    plt.show()

#     best_frequency = frequency[np.argmax(power)]





    # Create time series for folding
    #lc["time"]=Time(lc["time"].quantity, format="mjd")
    # #ts = TimeSeries(data=lc)
    
    #Show best fit model
    # plt.sca(axs[1])
    # ts_folded = ts.fold(period=1/f_peak, epoch_time=lc["time"][0])
    # plt.plot(ts_folded.time.jd, ts_folded['flux'], 'k.', markersize=4)
    # plt.xlabel('Time (days)')
    # plt.ylabel('Flux')
    
    # time_fit = np.linspace(0, 1)*uu.d
    # flux_fit = ls.model(time_fit*uu.d, f_peak)
    # plt.sca(axs[1])
    # plt.plot(time_fit,flux_fit,"ob")
    # plt.show()
    
    # fig = plt.figure(figsize=(14, 6))
    fig_lc, axs_lc = plt.subplots(1, 1, figsize=(14, 6), num="LC", clear=True)
    #axs_lc = axs_lc.flatten()
    vvis.plot_light_curve(tc_src, ax = axs_lc)
    #vvis.plot_light_curve(rg,rg_src_ids = rg_src_id, ax = axs_lc)

        #t_fit = lc["time"] #np.linspace(0, 1)*uu.d
    t_fit = np.linspace(t_min,t_max,1000)
    y_fit = ls.model(t_fit, f_peak)
    #print(t_fit, y_fit)
    axs_lc.plot(t_fit, y_fit,"b--", alpha=0.5)
    #axs[1].set_xlabel('Time (days)')
    #axs[1].set_ylabel('Flux')


    plt.show()


widgets.interactive(plot_LombScargle, rg_src_id=srcs_ids)

interactive(children=(Dropdown(description='rg_src_id', options=(8215, 32448, 54184, 98746, 151796, 166179, 17…

In [4]:
24*60/90 # MAximum Nr of orbits per day

16.0

In [5]:
1/(3600/4)

0.0011111111111111111